# 03. Экономические метрики для оценки классификаторов

## Цель
Реализовать экономические метрики вместо точности классификации для оценки полезности классификаторов рыночных зон.

## Задачи
1. Реализовать экономические метрики
2. Протестировать на данных BTC
3. Сравнить с традиционными метриками
4. Создать систему комплексной оценки


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sys
import os

# Добавляем путь к нашим модулям
sys.path.append('../evaluation')
sys.path.append('../classifiers')

from economic_metrics import EconomicMetrics
from classifiers import MZAClassifier

print("Библиотеки загружены успешно!")
print(f"Текущая директория: {os.getcwd()}")


In [ ]:
# Загружаем данные BTC для тестирования
print("=== ЗАГРУЗКА ДАННЫХ BTC ===")

# Загружаем данные из папки indicators/data_frames
file_path = '../../indicators/data_frames/df_btc_1h.csv'
df_btc = pd.read_csv(file_path)
df_btc['timestamps'] = pd.to_datetime(df_btc['timestamps'])
df_btc.set_index('timestamps', inplace=True)

print(f"✅ Загружено {len(df_btc)} записей BTC")
print(f"Период: {df_btc.index[0]} - {df_btc.index[-1]}")
print(f"Колонки: {list(df_btc.columns)}")

# Показываем первые 5 записей
print("\nПервые 5 записей:")
print(df_btc.head())


In [ ]:
# Тестируем MZA классификатор
print("=== ТЕСТИРОВАНИЕ MZA КЛАССИФИКАТОРА ===")

# Разделяем данные на train/test
train_size = int(len(df_btc) * 0.7)
train_data = df_btc.iloc[:train_size]
test_data = df_btc.iloc[train_size:]

print(f"Обучающая выборка: {len(train_data)} записей")
print(f"Тестовая выборка: {len(test_data)} записей")

# Создаем и обучаем MZA классификатор
mza = MZAClassifier()
mza.fit(train_data)
print("✅ MZA классификатор обучен")

# Получаем предсказания
predictions = mza.predict(test_data)
print(f"✅ Получено {len(predictions)} предсказаний")

# Статистика предсказаний
unique, counts = np.unique(predictions, return_counts=True)
print("\n📊 Статистика предсказаний:")
for regime, count in zip(unique, counts):
    regime_name = {-1: 'Медвежий', 0: 'Боковой', 1: 'Бычий'}.get(regime, f'Неизвестный_{regime}')
    percentage = count / len(predictions) * 100
    print(f"  {regime_name}: {count} ({percentage:.1f}%)")


In [ ]:
# Тестируем экономические метрики
print("=== ТЕСТИРОВАНИЕ ЭКОНОМИЧЕСКИХ МЕТРИК ===")

# Создаем объект для вычисления экономических метрик
economic_metrics = EconomicMetrics()

# Вычисляем экономические метрики
metrics = economic_metrics.calculate_economic_metrics(test_data, predictions)

print("📈 Экономические метрики:")
for key, value in metrics.items():
    print(f"  {key}: {value:.4f}")

# Вычисляем риск-скорректированные метрики
risk_metrics = economic_metrics.calculate_risk_adjusted_metrics(test_data, predictions)

print("\n📊 Риск-скорректированные метрики:")
for key, value in risk_metrics.items():
    print(f"  {key}: {value:.4f}")

# Вычисляем комплексный скор
comprehensive_score = economic_metrics.get_comprehensive_score(metrics)
print(f"\n🎯 Комплексный скор: {comprehensive_score:.4f}")


In [ ]:
# Визуализация экономических метрик
print("=== ВИЗУАЛИЗАЦИЯ ЭКОНОМИЧЕСКИХ МЕТРИК ===")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# График 1: Цена и предсказания
ax1 = axes[0, 0]
ax1.plot(test_data.index, test_data['close'], 'b-', linewidth=1, alpha=0.7, label='Цена BTC')

# Раскрашиваем по предсказаниям
colors = {-1: 'red', 0: 'gray', 1: 'green'}
for regime in [-1, 0, 1]:
    mask = predictions == regime
    if mask.any():
        regime_name = {-1: 'Медвежий', 0: 'Боковой', 1: 'Бычий'}[regime]
        ax1.scatter(test_data.index[mask], test_data['close'][mask], 
                   c=colors[regime], alpha=0.6, s=20, label=f'{regime_name} ({mask.sum()})')

ax1.set_title('MZA: Цена BTC и предсказания рыночных зон')
ax1.set_ylabel('Цена (USD)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# График 2: Доходность по режимам
ax2 = axes[0, 1]
returns = test_data['close'].pct_change().dropna()
regime_returns = [returns[predictions == regime].mean() for regime in [-1, 0, 1]]
regime_names = ['Медвежий', 'Боковой', 'Бычий']
colors_bars = ['red', 'gray', 'green']

bars = ax2.bar(regime_names, regime_returns, color=colors_bars, alpha=0.7)
ax2.set_title('Средняя доходность по режимам')
ax2.set_ylabel('Доходность')
ax2.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for bar, value in zip(bars, regime_returns):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.0001, 
             f'{value:.4f}', ha='center', va='bottom')

# График 3: Волатильность по режимам
ax3 = axes[1, 0]
regime_volatility = [returns[predictions == regime].std() for regime in [-1, 0, 1]]

bars = ax3.bar(regime_names, regime_volatility, color=colors_bars, alpha=0.7)
ax3.set_title('Волатильность по режимам')
ax3.set_ylabel('Волатильность')
ax3.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for bar, value in zip(bars, regime_volatility):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.0001, 
             f'{value:.4f}', ha='center', va='bottom')

# График 4: Коэффициент Шарпа по режимам
ax4 = axes[1, 1]
regime_sharpe = [returns[predictions == regime].mean() / returns[predictions == regime].std() 
                 if returns[predictions == regime].std() > 0 else 0 for regime in [-1, 0, 1]]

bars = ax4.bar(regime_names, regime_sharpe, color=colors_bars, alpha=0.7)
ax4.set_title('Коэффициент Шарпа по режимам')
ax4.set_ylabel('Коэффициент Шарпа')
ax4.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for bar, value in zip(bars, regime_sharpe):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("✅ Графики созданы")
